In [ ]:
#インポート
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm 
import openpyxl
import git
import os

#ローカルファイルの読み込み
import data_make as dm   #データを取得するためのファイル
import Input_form as Info #ファイル名の入力などを行うためのファイル

In [ ]:
#取得するデータの指定
repo_url, from_ver, to_ver, client_id, client_secret = Info.data_get()  #リポジトリのURL指定, 比較するブランチ, APIのid, KEY

In [ ]:
repo_path = '../repo_JSP'

if not os.path.exists(repo_path):
    os.mkdir(repo_path)
    print(repo_path + 'を作成しました.')

#すでに../repoにファイルが存在するならクローンしない
if os.listdir(repo_path) == []:
    #リポジトリのクローン
    git.Repo.clone_from(repo_url, repo_path, branch=to_ver)
    print('クローン成功')
else:
    print('すでに../repoにはファイルが存在しています')

In [ ]:
#ファイルの行数とファイルの依存関係の取得
java_line = dm.get_java_line(repo_path, repo_url, from_ver, to_ver, client_id, client_secret)
#コミット情報の取得
java_commit = dm.get_java_commit(repo_url, from_ver, to_ver, client_id, client_secret,java_line)
#変更行数の割合を取得
prob_list = dm.get_prob_list(java_line, java_commit)
#依存関係の取得
import_module, exception_module= dm.get_dependencies(repo_path)

In [ ]:
#結果の出力
print('import_module')
print('依存関係数: '+ str(len(import_module)))
print('exception_module')
print('例外依存関係数: '+ str(len(exception_module)))

print('java_line')
print('ファイル数: '+ str(len(java_line)))
print('java_commit')
print('コミットファイル数: ',str(len(java_commit)))
print('prob_list')
print(len(prob_list))

In [ ]:
#ファイル名のリストの作成
file_list = []
for i in java_line:
    file_list.append(i[0])


#java_lineにない例外依存関係を取り除く
new_exception_module = []
for i in tqdm(exception_module):
    for j in file_list:
        if i[1] in j:
            new_exception_module.append([i[0],j])
            
#java_lineにない依存関係を取り除く
new_import_module = []
for i in tqdm(import_module):
    for j in file_list:
        if i[1] in j:
            new_import_module.append([i[0],j])

In [ ]:
#無効グラフの作成
undirected_module = []
for i in tqdm(new_import_module):
    undirected_module.append([i[1],i[0]])
    
undirected_module = new_import_module + undirected_module
len(undirected_module)

In [ ]:
#例外処理のファイルの依存関係を取り除く
exclude_exception_module = []
for i in new_import_module:
    if i not in new_exception_module:
        exclude_exception_module.append(i)

In [ ]:
#無効グラフの作成
#例外処理のファイルの依存関係を取り除く
#無効グラフの作成
exclude_undirected_module = []
for i in tqdm(exclude_exception_module):
    exclude_undirected_module.append([i[1],i[0]])

exclude_undirected_module = exclude_exception_module + exclude_undirected_module
len(exclude_undirected_module)

In [ ]:
#空間スキャン検定に必要なファイルの読み込み
import CircularScan as cs

In [ ]:
scan_method, scan_type, graph_type, exception_select, threshold, iter_num, hop_count = Info.data_get2()
threshold = float(threshold)
iter_num = int(iter_num)
hop_count = int(hop_count)

In [ ]:
#依存関係の選択
if graph_type == '有効グラフ' and exception_select == "なし":
    java_module = exclude_exception_module #<--有向グラフ 例外処理なし
elif graph_type == '無向グラフ' and exception_select == "なし":
    java_module = exclude_undirected_module  #<-- 無向グラフ 例外処理なし
elif graph_type == '有効グラフ' and exception_select == "あり":
    java_module = new_import_module  #<-- 有向グラフ 例外処理あり
elif graph_type == '無向グラフ' and exception_select == "あり":
    java_module = undirected_module  #<-- 無向グラフ 例外処理あり
else:
    print('該当する依存関係なし')

#実行
if scan_method == 'repeat':
    result = cs.CS_run_repeat(java_line, java_commit, java_module, prob_list, scan_type, iter_num, hop_count, threshold)
elif scan_method == 'single':
    result = cs.CS_run_single(java_line, java_commit, java_module, prob_list, scan_type, iter_num, hop_count, threshold)

In [ ]:
result

In [ ]:
#結果の保存
result_path = '../result'

if not os.path.exists(result_path):
    os.mkdir(result_path)
    print(result_path + 'を作成しました.')

#保存ファイル名の決定
Excel_name = 'result_file'

#データフレームへの変換
df = pd.DataFrame(result, columns=['ファイル名','p値','LLR値'])

#エクセルデータとして結果を保存
df.to_excel(result_path+Excel_name+'.xlsx', sheet_name='new_sheet_name')

In [ ]:
#実験のログを保存
log = ['リポジトリ : ' + repo_url + '\n',
     'ブランチ : ' + from_ver + ' - ' + to_ver + '\n',
     '結果ファイル : ' + Excel_name + '.xlsx \n',
     'CSの関数 : '       + scan_method + '\n' , 
     '停止条件の種類 : ' + scan_type + '\n' ,
     'グラフの種類 : '   + graph_type + '\n', 
     '例外処理 : '       + exception_select + '\n' , 
     'しきい値 : '       + str(threshold) + '\n' ,
     'CSの実行回数 : '   + str(iter_num) + '\n',
     'CSのホップ数 : '   + str(hop_count)]

#書き込み
with open(result_path + '/' + Excel_name + '_result_log.txt', mode='w') as f:
    f.writelines(s)

with open(result_path + '/' + Excel_name + '_result_log.txt') as f:
    print(f.read())